In [1]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog as fd

# Import the data
root = tk.Tk()                     
root.withdraw()
root.attributes("-topmost", True)
root.focus_force()

Well_File_Path = fd.askopenfilename(filetypes=[('Well File', '.csv')])
df = pd.read_csv(Well_File_Path, on_bad_lines='skip')
df.head()

,Investigator,Alex Bajic,Time (s),Electrode,Amplitude (mV)
0,Recording Name,Before Hypoxia Plate 4,0.10848,A2_44,0.043
1,Description,Three months recording,2.40008,A1_75,0.019
2,NaN,NaN,2.47328,A1_75,0.020
3,Maestro Pro Settings,NaN,2.86056,A2_25,0.033
4,Original File Time,08/08/2024 15:27:55,2.98072,A2_25,0.029


In [2]:
import pandas as pd

# Assuming df is your DataFrame and it has a 'time' column
min_time = df['Time (s)'].min()
max_time = df['Time (s)'].max()

interval_start = 0
interval_end = max_time
interval_size = 1  # Adjust this based on your requirements

# Create intervals and bin the data
bins = np.arange(interval_start, interval_end + interval_size, interval_size)
df['Interval'] = pd.cut(df['Time (s)'], bins, right=False)

# List of all required channels from A1_11 to A1_88 and A2_11 to A2_88
required_columns = [f'A1_{i}' for i in range(11, 89)] + [f'A2_{i}' for i in range(11, 89)] + [f'A3_{i}' for i in range(11, 89)] + [f'B1_{i}' for i in range(11, 89)] + [f'B2_{i}' for i in range(11, 89)] + [f'B3_{i}' for i in range(11, 89)]

# Pivot the table to have electrodes as columns and intervals as rows
pivot_table = df.pivot_table(index='Interval', columns='Electrode', aggfunc='size', fill_value=0)

# Ensure all required columns are present and fill missing columns with 0s
pivot_table = pivot_table.reindex(columns=required_columns, fill_value=0)

# Create interval start and end times for the output
interval_table = pd.DataFrame({
    'Interval Start (S)': bins[:-1],
    'Interval End (S)': bins[1:]
})

# Merge the interval times with the pivot table
final_table = pd.concat([interval_table, pivot_table.reset_index(drop=True)], axis=1)

# Display the final result
final_table.head()

C:\Users\Lindsay\AppData\Local\Temp\ipykernel_19196\3552041330.py:19: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_table = df.pivot_table(index='Interval', columns='Electrode', aggfunc='size', fill_value=0)


,Interval Start (S),Interval End (S),A1_11,A1_12,A1_13,A1_14,A1_15,A1_16,A1_17,A1_18,...,B3_79,B3_80,B3_81,B3_82,B3_83,B3_84,B3_85,B3_86,B3_87,B3_88
0,0.0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,2.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2.0,3.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3.0,4.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4.0,5.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
final_table.to_csv(r'C:\Users\Lindsay\Github\MEA_data\final_table_3.csv', index=False)     